Modelo ML 

In [3]:
pip install xgboost


   ---------------------------------------- 0.0/99.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/99.8 MB 660.6 kB/s eta 0:02:31
   ---------------------------------------- 0.1/99.8 MB 1.7 MB/s eta 0:00:59
   ---------------------------------------- 0.6/99.8 MB 5.1 MB/s eta 0:00:20
    --------------------------------------- 1.7/99.8 MB 9.8 MB/s eta 0:00:10
   - -------------------------------------- 3.6/99.8 MB 16.6 MB/s eta 0:00:06
   -- ------------------------------------- 7.1/99.8 MB 26.8 MB/s eta 0:00:04
   ---- ----------------------------------- 10.4/99.8 MB 50.4 MB/s eta 0:00:02
   ----- ---------------------------------- 14.1/99.8 MB 81.8 MB/s eta 0:00:02
   ------ --------------------------------- 16.4/99.8 MB 73.1 MB/s eta 0:00:02
   -------- ------------------------------- 20.6/99.8 MB 65.6 MB/s eta 0:00:02
   ---------- ----------------------------- 25.5/99.8 MB 93.9 MB/s eta 0:00:01
   ------------ --------------------------- 30.8/99.8 MB 108.8 MB/s e

In [2]:
import pandas as pd
import xgboost as xgb
import numpy as np
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.compose import ColumnTransformer
from sklearn.metrics import accuracy_score
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression, Ridge, Lasso, LogisticRegression
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.decomposition import PCA
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn import set_config

In [3]:
# Dataframes
# Read .parquet file for epimediologic data from Chile
df_data_epi = pd.read_parquet('at_urg_respiratorio_semanal.parquet')

# Drop columns not interested
cols_interested = ['EstablecimientoGlosa','RegionGlosa','ComunaGlosa','TipoEstablecimiento','DependenciaAdministrativa',
                   'Latitud','Longitud','Anio','SemanaEstadistica','Causa','NumTotal']
df_model = df_data_epi[cols_interested]

# Preprocessing
# Drop NaNs
df_model.info()
df_model = df_model.dropna()
df_model.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2701467 entries, 0 to 2701466
Data columns (total 11 columns):
 #   Column                     Dtype 
---  ------                     ----- 
 0   EstablecimientoGlosa       object
 1   RegionGlosa                object
 2   ComunaGlosa                object
 3   TipoEstablecimiento        object
 4   DependenciaAdministrativa  object
 5   Latitud                    object
 6   Longitud                   object
 7   Anio                       int32 
 8   SemanaEstadistica          int32 
 9   Causa                      object
 10  NumTotal                   int32 
dtypes: int32(3), object(8)
memory usage: 195.8+ MB
<class 'pandas.core.frame.DataFrame'>
Index: 2699799 entries, 0 to 2701466
Data columns (total 11 columns):
 #   Column                     Dtype 
---  ------                     ----- 
 0   EstablecimientoGlosa       object
 1   RegionGlosa                object
 2   ComunaGlosa                object
 3   TipoEstablecimiento 

In [ ]:
# Exploration of features

#PCA

In [4]:
# Assign X and y 
Xcols_interested = ['EstablecimientoGlosa','RegionGlosa','ComunaGlosa','TipoEstablecimiento','DependenciaAdministrativa',
                    'Latitud','Longitud','Anio','SemanaEstadistica','Causa']
X_df= df_model[Xcols_interested]
y_df= df_model[['NumTotal']]

# Se borraron 
deleted = 2701467 - 2699799     #obtenido de los .info() de arriba
print(f'deleted: {deleted}')

deleted: 1668


Preprocesamiento y modelo

In [5]:
# Preprocessing for standarization and onehotencoding
# Identify categorical columns
categorical_features = X_df.select_dtypes(include=['object']).columns
numeric_features = X_df.select_dtypes(include=[np.number]).columns

# Instantiate a preprocessor 
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_features),
        ('cat', OneHotEncoder(), categorical_features)])
# Train test split
X_train, X_test, y_train, y_test = train_test_split(X_df, y_df, test_size= 0.2,random_state=27 )

# Transformation via preprocessor
X_train_s = preprocessor.fit_transform(X_train)
X_test_s = preprocessor.transform(X_test)

# Models
models = {
    'ridge': Ridge(),
    'lasso': Lasso(),
    'linear': LinearRegression(),
    'svr': SVR(),
    'random_forest': RandomForestRegressor(),
    'decision_tree': DecisionTreeRegressor(),
    'knn': KNeighborsRegressor() }
results = []

# Cros-validation
for name, model in models.items():
    kf = KFold(n_splits=6, random_state=27, shuffle=True)
    cv_results = cross_val_score(model, X_train_s, y_train, cv=kf)
    results.append(cv_results)
plt.boxplot(results, labels=models.keys())
plt.show()

c:\Users\Diego\anaconda3\Lib\site-packages\sklearn\utils\validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [ ]:
# Test performance, fit models
for name, model in models.items():
    model.fit(X_train_s, y_train)
    test_score = model.score(X_test_s, y_test)
    print("{} Test Set Accuracy: {}".format(name, test_score))

# Modelo XGBoost

# Lo siguiente son pruebas que quiero guardar

In [49]:
# Train test split
X_train, X_test, y_train, y_test = train_test_split(X_df, y_df, test_size= 0.2,random_state=27 )

# Steps of the pipeline
steps = [
    ('scaler', StandardScaler()),
    ('ridge', Ridge()),
    ('lasso', Lasso()),
    ('linear', LinearRegression()),
    ('svr', SVR()),
    ('random_forest', RandomForestRegressor()),
    ('decision_tree', DecisionTreeRegressor()),
    ('kn', KNeighborsRegressor())]

# Pipeline
pipeline = Pipeline(steps)

# Fit pipeline
pipeline.fit(X_train, y_train)

# Check pipeline detail or structure
set_config(display='diagram')       #revisar
pipeline

# Evaluate model 
scores = pipeline.score(X_test, y_test)
print('accuracy: %.3f' % scores)

# Predict on model
pred = pipeline.predict(X_test)
print(accuracy_score(y_test, pred))


TypeError: All intermediate steps should be transformers and implement fit and transform or be the string 'passthrough' 'Ridge()' (type <class 'sklearn.linear_model._ridge.Ridge'>) doesn't

# Apuntes mientras creo modelo
- debo hacer pipelines distintas si es que evaluo métricas distintas ? como si en algunos evaluo el rmse y r squared, o ocupo distintas variables para crear la curva roc ?
- cuantos tipos de modelo usaré ? todos los que tengo disponibles o algunos ?
- podría crear dummy variables para ciertas columnas, pero cuales ?